<a href="https://colab.research.google.com/github/zeeshanahmad54/H-M-Personalized-Fashion-Recommendations/blob/main/CSCI567_team72_H_and_M_fashion_Item_Similarities_with_prices_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and DF Setup

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 231 µs (started: 2022-05-04 23:12:34 +00:00)


In [ ]:
!pip install kaggle
!pip install sentence_transformers simcse

time: 8.06 s (started: 2022-05-04 23:12:34 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 922 ms (started: 2022-05-04 23:12:42 +00:00)


In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  
# # Then move kaggle.json into the folder where the API expects to find it.
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

time: 1.86 ms (started: 2022-05-04 23:12:43 +00:00)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering

from sentence_transformers import SentenceTransformer, util
import spacy
import pandas as pd
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import heapq


model = SentenceTransformer('stsb-roberta-large', device='cuda')

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)


time: 29.6 s (started: 2022-05-04 23:12:43 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


## Load DFs

In [ ]:
customer_df = pd.read_csv("/content/drive/MyDrive/CSCI_567_ML/Data/customers.csv")

time: 36.3 s (started: 2022-05-04 11:50:33 +00:00)


In [ ]:
article_df = pd.read_csv("/content/drive/MyDrive/CSCI_567_ML/Data/articles.csv")

In [ ]:
transaction_df = pd.read_csv("/content/drive/MyDrive/CSCI_567_ML/Data/transactions_train.csv")

time: 1min 3s (started: 2022-05-04 22:46:19 +00:00)


In [ ]:
images_folder = "/content/drive/MyDrive/CSCI_567_ML/Data/images"

time: 1.02 ms (started: 2022-05-04 12:55:20 +00:00)


## Explore Items

In [ ]:
image_feats = np.load("/content/drive/MyDrive/CSCI_567_ML/Data/H&M_image_encoding_dict_50.pkl", allow_pickle=True)#[()]

time: 1.43 s (started: 2022-05-04 07:19:31 +00:00)


### Helper functions

In [ ]:
from sentence_transformers import SentenceTransformer, util
import spacy
import pandas as pd
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import heapq


model = SentenceTransformer('stsb-roberta-large', device='cuda')

time: 23.6 s (started: 2022-05-04 07:19:32 +00:00)


In [ ]:
def description_embeddings(sentence):
  sentence = str(sentence)
  if sentence == str(np.nan):
    return np.nan
  else:
    # print(sentence)
    embeddings = model.encode(sentence)
    return embeddings   

time: 2.26 ms (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
corrected_image_feats = { k.split("/")[-1].split(".")[0][1:]: v for k, v in image_feats.items() }

time: 88.8 ms (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
product_id_to_image_fp = { k.split("/")[-1].split(".")[0][1:]: k for k, v in image_feats.items() }

time: 71.1 ms (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
# # verify
# print(product_id_to_image_fp["451243004"])
# print(corrected_image_feats["451243004"])

time: 954 µs (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
def image_fp(article_id):
  article_id = str(article_id)
  if article_id not in corrected_image_feats:
    # print("missing image for prod: ", article_id)
    return np.nan
  else:
    return product_id_to_image_fp[article_id]
    
  # print(article_id, type(article_id)) 

time: 2.4 ms (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
def image_embeddings(article_id):
  article_id = str(article_id)
  if article_id not in corrected_image_feats:
    # print("missing image for prod: ", article_id)
    return np.nan
  else:
    return corrected_image_feats[article_id]
    
  # print(article_id, type(article_id)) 

time: 2 ms (started: 2022-05-04 07:19:56 +00:00)


### Test code for visual similarity for a single category

In [ ]:
article_sample = article_df[article_df['product_type_name'] == "Bag"]

time: 10.3 ms (started: 2022-05-03 11:56:32 +00:00)


In [ ]:
# skip description similarity for now
# article_sample['desc_emb'] = article_sample['detail_desc'].progress_apply(description_embeddings)
article_sample['img_emb'] = article_sample['article_id'].progress_apply(image_embeddings)
article_sample['img_fp'] = article_sample['article_id'].progress_apply(image_fp)

100%|██████████| 1280/1280 [00:00<00:00, 293805.57it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


missing image for prod:  538923001
missing image for prod:  560783009
missing image for prod:  560783010
missing image for prod:  562298002
missing image for prod:  567435001
missing image for prod:  571716001
missing image for prod:  600441002


100%|██████████| 1280/1280 [00:00<00:00, 270178.10it/s]

time: 27.1 ms (started: 2022-05-03 11:09:54 +00:00)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# # don't fill NaNs with 0 before similarity. We will take care of NaNs later

# article_sample['desc_emb'].fillna(0, inplace=True)
# article_sample['img_emb'].fillna(0, inplace=True)

time: 6.02 ms (started: 2022-05-03 06:03:14 +00:00)


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
# We can only compute image similarity for non NaN entries

img_embs_not_na_df = article_sample[article_sample['img_emb'].notna()]
img_embs = list(img_embs_not_na_df['img_emb'])

# # Check how many entries were NaNs
# print(len(article_sample['img_emb']))
# print(len(img_embs_not_na_df))

time: 7.72 ms (started: 2022-05-03 11:09:56 +00:00)


In [ ]:
# img_embs_not_na_df.head()

In [ ]:
cos_sim_img = util.cos_sim(img_embs, img_embs)
print("Cosine-Similarity img:", cos_sim_img)

Cosine-Similarity img: tensor([[1.0000, 0.7970, 0.6525,  ..., 0.7510, 0.7153, 0.7552],
        [0.7970, 1.0000, 0.7151,  ..., 0.7054, 0.7458, 0.6692],
        [0.6525, 0.7151, 1.0000,  ..., 0.7137, 0.6920, 0.6494],
        ...,
        [0.7510, 0.7054, 0.7137,  ..., 1.0000, 0.7430, 0.7145],
        [0.7153, 0.7458, 0.6920,  ..., 0.7430, 1.0000, 0.7431],
        [0.7552, 0.6692, 0.6494,  ..., 0.7145, 0.7431, 1.0000]])
time: 411 ms (started: 2022-05-03 11:09:57 +00:00)


In [ ]:
num = 13 # includes the item itself so we take 12+1 

# for idx, row in enumerate(cos_sim_img):
#   topN_sim = heapq.nlargest(num, row)
#   # print(top4_sim)

topN_sim_idx = np.argpartition(cos_sim_img, -num, axis=1)[:, -num:]
# print(topN_sim_idx)

time: 21.3 ms (started: 2022-05-03 11:09:58 +00:00)


In [ ]:
# topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]
# topN_sim_classes_arr = np.asarray(topN_sim_classes)
# print(topN_sim_classes_arr.shape)
# img_embs_not_na_df['top_sim'] = topN_sim_classes

# img_embs_not_na_df = img_embs_not_na_df.assign(column_new_1=np.nan, column_new_2='dogs', column_new_3=3)


(1273, 12)
time: 349 ms (started: 2022-05-03 10:38:51 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# For top 12 similar products, add 12 new columns in article_df
topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]
topN_sim_classes_arr = np.asarray(topN_sim_classes)
print(topN_sim_classes_arr.shape)
# test[:,0]
print(topN_sim_classes_arr[:,-1])
img_embs_not_na_df['top_sim'] = topN_sim_classes

img_embs_not_na_df = img_embs_not_na_df.assign(sim_1=topN_sim_classes_arr[:,-1], sim_2=topN_sim_classes_arr[:,-2], sim_3=topN_sim_classes_arr[:,-3], sim_4=topN_sim_classes_arr[:,-4], sim_5=topN_sim_classes_arr[:,-5], sim_6=topN_sim_classes_arr[:,-6], \
                                               sim_7=topN_sim_classes_arr[:,-7], sim_8=topN_sim_classes_arr[:,-8], sim_9=topN_sim_classes_arr[:,-9], sim_10=topN_sim_classes_arr[:,-10], sim_11=topN_sim_classes_arr[:,-11], sim_12=topN_sim_classes_arr[:,-12])

# img_embs_not_na_df = img_embs_not_na_df.assign(sim_1_prod=img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']])
# img_embs_not_na_df = img_embs_not_na_df.assign(sim_1_prod=img_embs_not_na_df['article_id'].iloc[int(img_embs_not_na_df['sim_1'])], sim_2_prod=topN_sim_classes_arr[:,-2], sim_3_prod=topN_sim_classes_arr[:,-3], sim_4_prod=topN_sim_classes_arr[:,-4], sim_5_prod=topN_sim_classes_arr[:,-5], sim_6_prod=topN_sim_classes_arr[:,-6], \
#                                                sim_7_prod=topN_sim_classes_arr[:,-7], sim_8_prod=topN_sim_classes_arr[:,-8], sim_9_prod=topN_sim_classes_arr[:,-9], sim_10_prod=topN_sim_classes_arr[:,-10], sim_11_prod=topN_sim_classes_arr[:,-11], sim_12_prod=topN_sim_classes_arr[:,-12])

# img_embs_not_na_df['article_id'].iloc[int(prod)]


# img_embs_not_na_df[['sim_1', 'sim_2', 'sim_3', 'sim_4', 'sim_5', 'sim_6', 'sim_7', 'sim_8', 'sim_9', 'sim_10', 'sim_11', 'sim_12']] = img_embs_not_na_df.apply(top12similiar_prods, axis=1)

(1273, 12)
[ 709  118 1159 ...  505  714  337]
time: 117 ms (started: 2022-05-03 11:21:58 +00:00)


In [ ]:
# img_embs_not_na_df = img_embs_not_na_df.assign(sim_1_prod=img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']])
sim_1_prod = img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']]
print(type(sim_1_prod))
img_embs_not_na_df['sim_1_prod'] = list(sim_1_prod)

<class 'pandas.core.series.Series'>
time: 6.35 ms (started: 2022-05-03 11:24:28 +00:00)


In [ ]:
# Verify!
img_embs_not_na_df.head()

# least sim to most for Bags
#[tensor(112), tensor(710), tensor(734), tensor(505), tensor(559), tensor(15), tensor(138), tensor(410), tensor(66), tensor(118), tensor(76), tensor(709)]

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,sim_3,sim_4,sim_5,sim_6,sim_7,sim_8,sim_9,sim_10,sim_11,sim_12
438,241436001,241436,Fix it tote bag,66,Bag,Accessories,1010016,Solid,9,Black,...,118,66,410,138,15,559,505,734,710,112
439,241436006,241436,Fix it tote bag,66,Bag,Accessories,1010016,Solid,12,Light Beige,...,258,71,66,671,212,981,137,11,9,710
723,289834016,289834,FANTASTIC SHOPPER,66,Bag,Accessories,1010014,Placement print,11,Off White,...,1152,236,393,518,1231,300,665,1125,897,398
1583,360295023,360295,Kardashian shopper,66,Bag,Accessories,1010016,Solid,8,Dark Grey,...,733,975,94,1247,121,17,802,845,1026,1203
1883,380894037,380894,ChloÃ© cosmetic bag,66,Bag,Accessories,1010011,Metallic,61,Light Purple,...,1170,142,961,817,518,623,18,444,143,489


time: 19.4 ms (started: 2022-05-03 10:42:59 +00:00)


In [ ]:
article_df = pd.merge(article_df,img_embs_not_na_df[['article_id', 'sim_1', 'sim_2', 'sim_3', 'sim_4', 'sim_5', 'sim_6', 'sim_7', 'sim_8', 'sim_9', 'sim_10', 'sim_11', 'sim_12']],on='article_id', how='left')

time: 200 ms (started: 2022-05-03 10:50:28 +00:00)


### Visual similarity for all categories

In [ ]:
# Decide product types to compute similarity among
pd.options.display.max_rows = None
article_df.groupby(['product_group_name', 'product_type_name']).count()['article_id']

product_group_name     product_type_name       
Accessories            Accessories set             7    
                       Alice band                  6    
                       Baby Bib                    3    
                       Bag                         1280 
                       Beanie                      56   
                       Belt                        458  
                       Bracelet                    180  
                       Braces                      3    
                       Bucket hat                  7    
                       Cap                         13   
                       Cap/peaked                  573  
                       Dog Wear                    20   
                       Earring                     1159 
                       Earrings                    11   
                       Eyeglasses                  2    
                       Felt hat                    10   
                       Giftbox          

time: 105 ms (started: 2022-05-04 05:10:23 +00:00)


In [ ]:
product_group_name = list(np.sort(article_df['product_group_name'].unique()))
prod_group_wise_item_count  = article_df.groupby(['product_group_name']).count()['article_id'].to_dict()
# product_group_name

time: 96 ms (started: 2022-05-04 05:10:23 +00:00)


In [ ]:
print(product_group_name)
product_group_name.pop(7)
print(product_group_name)

['Accessories', 'Bags', 'Cosmetic', 'Fun', 'Furniture', 'Garment Full body', 'Garment Lower body', 'Garment Upper body', 'Garment and Shoe care', 'Interior textile', 'Items', 'Nightwear', 'Shoes', 'Socks & Tights', 'Stationery', 'Swimwear', 'Underwear', 'Underwear/nightwear', 'Unknown']
['Accessories', 'Bags', 'Cosmetic', 'Fun', 'Furniture', 'Garment Full body', 'Garment Lower body', 'Garment and Shoe care', 'Interior textile', 'Items', 'Nightwear', 'Shoes', 'Socks & Tights', 'Stationery', 'Swimwear', 'Underwear', 'Underwear/nightwear', 'Unknown']
time: 3.53 ms (started: 2022-05-04 05:10:23 +00:00)


In [ ]:
prod_group_wise_item_count

{'Accessories': 11158,
 'Bags': 25,
 'Cosmetic': 49,
 'Fun': 2,
 'Furniture': 13,
 'Garment Full body': 13292,
 'Garment Lower body': 19812,
 'Garment Upper body': 42741,
 'Garment and Shoe care': 9,
 'Interior textile': 3,
 'Items': 17,
 'Nightwear': 1899,
 'Shoes': 5283,
 'Socks & Tights': 2442,
 'Stationery': 5,
 'Swimwear': 3127,
 'Underwear': 5490,
 'Underwear/nightwear': 54,
 'Unknown': 121}

time: 4.59 ms (started: 2022-05-04 05:10:23 +00:00)


In [ ]:
# Everything combined for an item
import gc

# article_df_new = article_df.copy()

appended_data = []

for item_group in tqdm(product_group_name):
  #1. Create a subset of the df for a product type
  print(item_group)
  article_sample = article_df[article_df['product_group_name'] == item_group]

  #2. Add image embeddings and image file paths for each prod
  # skip description similarity for now
  # article_sample['desc_emb'] = article_sample['detail_desc'].progress_apply(description_embeddings)
  article_sample['img_emb'] = article_sample['article_id'].progress_apply(image_embeddings)
  article_sample['img_fp'] = article_sample['article_id'].progress_apply(image_fp)

  #3. Remove NaN values from subset df
  # We can only compute image similarity for non NaN entries

  img_embs_not_na_df = article_sample[article_sample['img_emb'].notna()]
  img_embs = list(img_embs_not_na_df['img_emb']) #Create a list to compute cosine sim

  #4. compute cosine similarity among all products images in the subset df
  cos_sim_img = util.cos_sim(img_embs, img_embs)
  # print("Cosine-Similarity img:", cos_sim_img)

  #5. Obtain top N similar instances
  # num = 13 # includes the item itself so we take 12+1 
  num = min(13, prod_group_wise_item_count[item_group])
  topN_sim_idx = np.argpartition(cos_sim_img, -num, axis=1)[:, -num:]

  #6. Add 12 cols for the top 12 most similar products 
  topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]
  topN_sim_classes_arr = np.asarray(topN_sim_classes)
  print(f"\n{topN_sim_classes_arr.shape}")
  if topN_sim_classes_arr.shape[1] < 12:
    nan_col = np.empty((topN_sim_classes_arr.shape[0],12-topN_sim_classes_arr.shape[1]))
    nan_col.fill(np.nan)
    topN_sim_classes_arr = np.c_[topN_sim_classes_arr,nan_col]
    print(f"\nReshaping with NaNs to accomodate for less # of products in category: {item_group}")
    print(topN_sim_classes_arr)


  img_embs_not_na_df = img_embs_not_na_df.assign(sim_1=topN_sim_classes_arr[:,-1], sim_2=topN_sim_classes_arr[:,-2], sim_3=topN_sim_classes_arr[:,-3], sim_4=topN_sim_classes_arr[:,-4], sim_5=topN_sim_classes_arr[:,-5], sim_6=topN_sim_classes_arr[:,-6], \
                                                sim_7=topN_sim_classes_arr[:,-7], sim_8=topN_sim_classes_arr[:,-8], sim_9=topN_sim_classes_arr[:,-9], sim_10=topN_sim_classes_arr[:,-10], sim_11=topN_sim_classes_arr[:,-11], sim_12=topN_sim_classes_arr[:,-12])
  
  img_embs_not_na_df['sim_1_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_2_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_3_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_4_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_5_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_6_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_7_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_8_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_9_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_10_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_11_prod'] = pd.Series([], dtype=int)
  img_embs_not_na_df['sim_12_prod'] = pd.Series([], dtype=int)  

  try:
    # img_embs_not_na_df.assign(sim_1_prod=list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']]))
    img_embs_not_na_df['sim_1_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']])
  except:
    img_embs_not_na_df['sim_1_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_2_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_2']])
  except:
    img_embs_not_na_df['sim_2_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_3_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_3']])
  except:
    img_embs_not_na_df['sim_3_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_4_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_4']])
  except:
    img_embs_not_na_df['sim_4_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_5_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_5']])
  except:
    img_embs_not_na_df['sim_5_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_6_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_6']])
  except:
    img_embs_not_na_df['sim_6_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_7_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_7']])
  except:
    img_embs_not_na_df['sim_7_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_8_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_8']])
  except:
    img_embs_not_na_df['sim_8_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_9_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_9']])
  except:
    img_embs_not_na_df['sim_9_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_10_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_10']])
  except:
    img_embs_not_na_df['sim_10_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_11_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_11']])
  except:
    img_embs_not_na_df['sim_11_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_12_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_12']])
  except:
    img_embs_not_na_df['sim_12_prod'] = np.nan

  #7. Merge the new 12 cols with original article_df
  # article_df = pd.merge(article_df,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
  # article_df_new = pd.merge(article_df_new,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', suffixes=('', '_y'))
  # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

  # article_df_new = article_df_new.merge(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']], left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
  # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
  appended_data.append(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']])


  del img_embs_not_na_df
  gc.collect()

  0%|          | 0/18 [00:00<?, ?it/s]

Accessories



100%|██████████| 11158/11158 [00:00<00:00, 511107.22it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()

100%|██████████| 11158/11158 [00:00<00:00, 432772.74it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(11005, 12)


  6%|▌         | 1/18 [00:07<02:07,  7.49s/it]

...

Unknown



100%|██████████| 121/121 [00:00<00:00, 85396.40it/s]

100%|██████████| 121/121 [00:00<00:00, 124848.90it/s]



(121, 12)


100%|██████████| 18/18 [00:50<00:00,  2.80s/it]

time: 50.6 s (started: 2022-05-04 05:15:47 +00:00)


In [ ]:
# special handling for garment_upper_body
garment_upper_body = article_df[article_df['product_group_name'] == "Garment Upper body"].groupby(['product_type_name']).count()['article_id']
garment_upper_body_type_wise_item_count  = garment_upper_body.to_dict()
garment_upper_body_list = list(garment_upper_body.keys())

time: 57.4 ms (started: 2022-05-04 05:16:39 +00:00)


In [ ]:
# Everything combined for an item
import gc




# article_df_new = article_df.copy()

for item_type in tqdm(garment_upper_body_list):
  #1. Create a subset of the df for a product type
  print(item_type)
  article_sample = article_df[article_df['product_type_name'] == item_type]

  #2. Add image embeddings and image file paths for each prod
  # skip description similarity for now
  # article_sample['desc_emb'] = article_sample['detail_desc'].progress_apply(description_embeddings)
  article_sample['img_emb'] = article_sample['article_id'].progress_apply(image_embeddings)
  article_sample['img_fp'] = article_sample['article_id'].progress_apply(image_fp)

  #3. Remove NaN values from subset df
  # We can only compute image similarity for non NaN entries

  img_embs_not_na_df = article_sample[article_sample['img_emb'].notna()]
  img_embs = list(img_embs_not_na_df['img_emb']) #Create a list to compute cosine sim

  #4. compute cosine similarity among all products images in the subset df
  cos_sim_img = util.cos_sim(img_embs, img_embs)
  # print("Cosine-Similarity img:", cos_sim_img)

  #5. Obtain top N similar instances
  # num = 13 # includes the item itself so we take 12+1 
  num = min(13, garment_upper_body_type_wise_item_count[item_type])
  topN_sim_idx = np.argpartition(cos_sim_img, -num, axis=1)[:, -num:]

  #6. Add 12 cols for the top 12 most similar products 
  topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]
  topN_sim_classes_arr = np.asarray(topN_sim_classes)
  print(f"\n{topN_sim_classes_arr.shape}")
  if topN_sim_classes_arr.shape[1] < 12:
    nan_col = np.empty((topN_sim_classes_arr.shape[0],12-topN_sim_classes_arr.shape[1]))
    nan_col.fill(np.nan)
    topN_sim_classes_arr = np.c_[topN_sim_classes_arr,nan_col]
    print(f"\nReshaping with NaNs to accomodate for less # of products in category: {item_type}")
    print(topN_sim_classes_arr)


  img_embs_not_na_df = img_embs_not_na_df.assign(sim_1=topN_sim_classes_arr[:,-1], sim_2=topN_sim_classes_arr[:,-2], sim_3=topN_sim_classes_arr[:,-3], sim_4=topN_sim_classes_arr[:,-4], sim_5=topN_sim_classes_arr[:,-5], sim_6=topN_sim_classes_arr[:,-6], \
                                                sim_7=topN_sim_classes_arr[:,-7], sim_8=topN_sim_classes_arr[:,-8], sim_9=topN_sim_classes_arr[:,-9], sim_10=topN_sim_classes_arr[:,-10], sim_11=topN_sim_classes_arr[:,-11], sim_12=topN_sim_classes_arr[:,-12])

  try:
    # img_embs_not_na_df.assign(sim_1_prod=list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']]))
    img_embs_not_na_df['sim_1_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']])
  except:
    img_embs_not_na_df['sim_1_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_2_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_2']])
  except:
    img_embs_not_na_df['sim_2_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_3_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_3']])
  except:
    img_embs_not_na_df['sim_3_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_4_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_4']])
  except:
    img_embs_not_na_df['sim_4_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_5_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_5']])
  except:
    img_embs_not_na_df['sim_5_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_6_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_6']])
  except:
    img_embs_not_na_df['sim_6_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_7_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_7']])
  except:
    img_embs_not_na_df['sim_7_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_8_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_8']])
  except:
    img_embs_not_na_df['sim_8_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_9_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_9']])
  except:
    img_embs_not_na_df['sim_9_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_10_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_10']])
  except:
    img_embs_not_na_df['sim_10_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_11_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_11']])
  except:
    img_embs_not_na_df['sim_11_prod'] = np.nan
  try:
    img_embs_not_na_df['sim_12_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_12']])
  except:
    img_embs_not_na_df['sim_12_prod'] = np.nan

  #7. Merge the new 12 cols with original article_df
  # article_df = pd.merge(article_df,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
  # article_df_new = pd.merge(article_df_new,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', suffixes=('', '_y'))
  # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

  # article_df_new = article_df_new.merge(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']], left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
  # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
  appended_data.append(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']])


  del img_embs_not_na_df
  gc.collect()

  0%|          | 0/15 [00:00<?, ?it/s]

Blazer



100%|██████████| 1110/1110 [00:00<00:00, 360821.32it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 1110/1110 [00:00<00:00, 362930.89it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(1108, 12)


  7%|▋         | 1/15 [00:00<00:09,  1.43it/s]

...

Vest top



100%|██████████| 2991/2991 [00:00<00:00, 493670.84it/s]

100%|██████████| 2991/2991 [00:00<00:00, 412032.82it/s]



(2979, 12)


100%|██████████| 15/15 [00:26<00:00,  1.74s/it]

time: 26.2 s (started: 2022-05-04 05:16:39 +00:00)


In [ ]:
article_df_new = pd.concat(appended_data)
print(len(article_df_new))

105100
time: 15.7 ms (started: 2022-05-04 05:17:05 +00:00)


In [ ]:
# for col in article_df.columns:
#   print(col)

time: 844 µs (started: 2022-05-03 13:57:01 +00:00)


In [ ]:
for col in article_df_new.columns:
  print(col)

article_id
sim_1_prod
sim_2_prod
sim_3_prod
sim_4_prod
sim_5_prod
sim_6_prod
sim_7_prod
sim_8_prod
sim_9_prod
sim_10_prod
sim_11_prod
sim_12_prod
time: 2.72 ms (started: 2022-05-04 06:41:28 +00:00)


In [ ]:
print(len(article_df))
print(len(article_df_new))

105542
105100
time: 3.27 ms (started: 2022-05-04 05:17:05 +00:00)


In [ ]:
# article_df = pd.merge(article_df,article_df_new[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
# article_df = pd.merge(article_df,article_df_new[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id')
article_df = article_df.merge(article_df_new, on='article_id', how='outer')


time: 210 ms (started: 2022-05-04 05:17:05 +00:00)


In [ ]:
print(len(article_df))

105542
time: 4.36 ms (started: 2022-05-04 06:41:16 +00:00)


In [ ]:
print(article_df.head())

   article_id  product_code          prod_name  product_type_no  \
0  108775015   108775        Strap top          253               
1  108775044   108775        Strap top          253               
2  108775051   108775        Strap top (1)      253               
3  110065001   110065        OP T-shirt (Idro)  306               
4  110065002   110065        OP T-shirt (Idro)  306               

  product_type_name  product_group_name  graphical_appearance_no  \
0  Vest top          Garment Upper body  1010016                   
1  Vest top          Garment Upper body  1010016                   
2  Vest top          Garment Upper body  1010017                   
3  Bra               Underwear           1010016                   
4  Bra               Underwear           1010016                   

  graphical_appearance_name  colour_group_code colour_group_name  \
0  Solid                     9                  Black              
1  Solid                     10                 Whit

In [ ]:
article_df.to_csv("/content/drive/MyDrive/CSCI_567_ML/Data/articles_with_top12_similar_images.csv")

time: 2.58 s (started: 2022-05-03 14:04:18 +00:00)


In [ ]:
# # Delete the instances of the index in sim matrix if it matches with row index
# topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]

# for idx, row in enumerate(topN_sim_classes[:5]):
#   print(f"prod_id: {img_embs_not_na_df['article_id'].iloc[idx]}, fp: {img_embs_not_na_df['img_fp'].iloc[idx]}, prod: {img_embs_not_na_df['colour_group_name'].iloc[idx]}, {img_embs_not_na_df['prod_name'].iloc[idx]}, desc: {img_embs_not_na_df['detail_desc'].iloc[idx]}")
#   print(row)
#   for idx_prod, prod in enumerate(row[::-1]):
#     print(f"Sim_{idx_prod} - prod_id: {img_embs_not_na_df['article_id'].iloc[int(prod)]}, fp: {img_embs_not_na_df['img_fp'].iloc[int(prod)]}, prod: {img_embs_not_na_df['colour_group_name'].iloc[idx]}, {img_embs_not_na_df['prod_name'].iloc[int(prod)]}, desc: {img_embs_not_na_df['detail_desc'].iloc[int(prod)]}")
#   print("\n")

In [ ]:
desc_embs = [x for x in article_sample['desc_emb']]

time: 4.98 ms (started: 2022-05-03 05:44:31 +00:00)


In [ ]:
print(desc_embs)

In [ ]:
# # cos_sim_text = util.cos_sim(desc_embs, desc_embs)
# cos_sim_img = util.cos_sim(article_sample['img_emb'], article_sample['img_emb'])
# # print("Cosine-Similarity text:", cos_sim_text)
# print("Cosine-Similarity img:", cos_sim_img)

In [ ]:
# # image embedding length
# print(len(article_sample['img_emb'].iloc[0]))

# # description embedding length
# print(len(article_sample['desc_emb'].iloc[0]))

2048
time: 3.35 ms (started: 2022-05-03 05:28:09 +00:00)


### Visual similarity for all categories based on price

In [ ]:
# Decide product types to compute similarity among
pd.options.display.max_rows = None
article_df.groupby(['product_group_name', 'product_type_name']).count()['article_id']

In [ ]:
product_group_name = list(np.sort(article_df['product_group_name'].unique()))
prod_group_wise_item_count  = article_df.groupby(['product_group_name']).count()['article_id'].to_dict()
print(product_group_name)
product_group_name.pop(7)
print(product_group_name)

time: 85.3 ms (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
for col in transaction_df.columns:
  print(col)

t_dat
customer_id
article_id
price
sales_channel_id
time: 1.03 ms (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
# min_price_item = transaction_df.nlargest(2,['price'])['article_id']

697511001
time: 2.34 s (started: 2022-05-04 07:19:56 +00:00)


In [ ]:
transactions_by_article_id = transaction_df.groupby('article_id').mean()
print(len(transactions_by_article_id))

time: 2.08 s (started: 2022-05-04 07:31:08 +00:00)


In [ ]:
transactions_by_article_id['price'].head()

article_id
108775015   0.0081
108775044   0.0081
108775051   0.0050
110065001   0.0202
110065002   0.0182
Name: price, dtype: float64

time: 6.39 ms (started: 2022-05-04 07:37:12 +00:00)


In [ ]:
articles_for_merge = article_df[['article_id', 'prod_name', 'product_type_name', 'product_group_name', 'colour_group_name', 'index_name', 'detail_desc']]
print(len(articles_for_merge))

time: 8.57 ms (started: 2022-05-04 07:38:49 +00:00)


In [ ]:
articles_with_prices = articles_for_merge.merge(transactions_by_article_id['price'], on='article_id', how="outer")
articles_with_prices = articles_with_prices.sort_values(by='price', ascending=False)
print(len(articles_with_prices))

time: 35.2 ms (started: 2022-05-04 07:39:44 +00:00)


In [ ]:
articles_with_prices_not_na = articles_with_prices.dropna()

time: 46.7 ms (started: 2022-05-04 09:19:00 +00:00)


In [ ]:
print(len(articles_with_prices))
print(len(articles_with_prices_not_na))

105542
104131
time: 1.58 ms (started: 2022-05-04 10:24:26 +00:00)


In [ ]:
# Everything combined for an item
import gc

# article_df_new = article_df.copy()

appended_data = []

for item_group in tqdm(product_group_name):
  #1. Create a subset of the df for a product type
  print(item_group)
  # article_sample = article_df[article_df['product_group_name'] == item_group]
  print(len(articles_with_prices))
  article_sample = articles_with_prices[articles_with_prices['product_group_name'] == item_group]
  print(len(article_sample))
  #2. Add image embeddings and image file paths for each prod
  # skip description similarity for now
  # article_sample['desc_emb'] = article_sample['detail_desc'].progress_apply(description_embeddings)
  article_sample['img_emb'] = article_sample['article_id'].progress_apply(image_embeddings)
  article_sample['img_fp'] = article_sample['article_id'].progress_apply(image_fp)

  #3. Remove NaN values from subset df
  # We can only compute image similarity for non NaN entries

  articles_sample_with_prices_not_na = article_sample.dropna()

  df_where_img_embs_not_na = articles_sample_with_prices_not_na[articles_sample_with_prices_not_na['img_emb'].notna()]


  # articles_sample_with_prices_not_na = img_embs_not_na_df.dropna()
  kmeans = KMeans(n_clusters=2, random_state=0).fit(df_where_img_embs_not_na['price'].values.reshape(-1, 1))
  df_where_img_embs_not_na["price_cluster"] = kmeans.labels_

  exp_df = df_where_img_embs_not_na[df_where_img_embs_not_na['price_cluster'] == 1]
  cheap_df = df_where_img_embs_not_na[df_where_img_embs_not_na['price_cluster'] == 0]
  print(f"\n\nexp_df len: {len(exp_df)}, cheap_df len: {len(cheap_df)}")

  for img_embs_not_na_df in [exp_df, cheap_df]:
    img_embs = list(img_embs_not_na_df['img_emb'])


    # img_embs = list(img_embs_not_na_df['img_emb']) #Create a list to compute cosine sim

    #4. compute cosine similarity among all products images in the subset df
    cos_sim_img = util.cos_sim(img_embs, img_embs)
    # print("Cosine-Similarity img:", cos_sim_img)

    #5. Obtain top N similar instances
    # num = 13 # includes the item itself so we take 12+1 
    num = min(13, len(exp_df), len(cheap_df))
    topN_sim_idx = np.argpartition(cos_sim_img, -num, axis=1)[:, -num:]

    #6. Add 12 cols for the top 12 most similar products 
    topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]
    topN_sim_classes_arr = np.asarray(topN_sim_classes)
    print(f"\n{topN_sim_classes_arr.shape}")
    if topN_sim_classes_arr.shape[1] < 12:
      nan_col = np.empty((topN_sim_classes_arr.shape[0],12-topN_sim_classes_arr.shape[1]))
      nan_col.fill(np.nan)
      topN_sim_classes_arr = np.c_[topN_sim_classes_arr,nan_col]
      print(f"\nReshaping with NaNs to accomodate for less # of products in category: {item_group}")
      print(topN_sim_classes_arr)


    img_embs_not_na_df = img_embs_not_na_df.assign(sim_1=topN_sim_classes_arr[:,-1], sim_2=topN_sim_classes_arr[:,-2], sim_3=topN_sim_classes_arr[:,-3], sim_4=topN_sim_classes_arr[:,-4], sim_5=topN_sim_classes_arr[:,-5], sim_6=topN_sim_classes_arr[:,-6], \
                                                  sim_7=topN_sim_classes_arr[:,-7], sim_8=topN_sim_classes_arr[:,-8], sim_9=topN_sim_classes_arr[:,-9], sim_10=topN_sim_classes_arr[:,-10], sim_11=topN_sim_classes_arr[:,-11], sim_12=topN_sim_classes_arr[:,-12])
    
    try:
      # img_embs_not_na_df.assign(sim_1_prod=list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']]))
      img_embs_not_na_df['sim_1_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']])
    except:
      img_embs_not_na_df['sim_1_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_2_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_2']])
    except:
      img_embs_not_na_df['sim_2_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_3_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_3']])
    except:
      img_embs_not_na_df['sim_3_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_4_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_4']])
    except:
      img_embs_not_na_df['sim_4_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_5_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_5']])
    except:
      img_embs_not_na_df['sim_5_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_6_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_6']])
    except:
      img_embs_not_na_df['sim_6_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_7_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_7']])
    except:
      img_embs_not_na_df['sim_7_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_8_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_8']])
    except:
      img_embs_not_na_df['sim_8_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_9_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_9']])
    except:
      img_embs_not_na_df['sim_9_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_10_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_10']])
    except:
      img_embs_not_na_df['sim_10_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_11_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_11']])
    except:
      img_embs_not_na_df['sim_11_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_12_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_12']])
    except:
      img_embs_not_na_df['sim_12_prod'] = np.nan

    #7. Merge the new 12 cols with original article_df
    # article_df = pd.merge(article_df,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
    # article_df_new = pd.merge(article_df_new,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', suffixes=('', '_y'))
    # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

    # article_df_new = article_df_new.merge(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']], left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
    # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
    appended_data.append(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']])


  del img_embs_not_na_df
  gc.collect()

  0%|          | 0/18 [00:00<?, ?it/s]

Accessories
105542
11158



100%|██████████| 11158/11158 [00:00<00:00, 511677.21it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 11158/11158 [00:00<00:00, 556672.86it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




exp_df len: 672, cheap_df len: 10181

(672, 12)

(10181, 12)


  6%|▌         | 1/18 [00:10<02:52, 10.16s/it]

...

Unknown
105542
121



100%|██████████| 121/121 [00:00<00:00, 132960.65it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 121/121 [00:00<00:00, 128908.00it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




exp_df len: 15, cheap_df len: 98

(15, 12)

(98, 12)


100%|██████████| 18/18 [01:00<00:00,  3.38s/it]

time: 1min (started: 2022-05-04 10:02:29 +00:00)


In [ ]:
# special handling for garment_upper_body
garment_upper_body = articles_with_prices[articles_with_prices['product_group_name'] == "Garment Upper body"].groupby(['product_type_name']).count()['article_id']
garment_upper_body_type_wise_item_count  = garment_upper_body.to_dict()
garment_upper_body_list = list(garment_upper_body.keys())

time: 34.9 ms (started: 2022-05-04 10:03:30 +00:00)


In [ ]:
# Everything combined for an item
import gc

# article_df_new = article_df.copy()

for item_type in tqdm(garment_upper_body_list):
  #1. Create a subset of the df for a product type
  print(item_type)
  # article_sample = article_df[article_df['product_group_name'] == item_group]
  print(len(articles_with_prices))
  article_sample = articles_with_prices[articles_with_prices['product_type_name'] == item_type]
  print(len(article_sample))
  #2. Add image embeddings and image file paths for each prod
  # skip description similarity for now
  # article_sample['desc_emb'] = article_sample['detail_desc'].progress_apply(description_embeddings)
  article_sample['img_emb'] = article_sample['article_id'].progress_apply(image_embeddings)
  article_sample['img_fp'] = article_sample['article_id'].progress_apply(image_fp)

  #3. Remove NaN values from subset df
  # We can only compute image similarity for non NaN entries

  articles_sample_with_prices_not_na = article_sample.dropna()

  df_where_img_embs_not_na = articles_sample_with_prices_not_na[articles_sample_with_prices_not_na['img_emb'].notna()]


  # articles_sample_with_prices_not_na = img_embs_not_na_df.dropna()
  kmeans = KMeans(n_clusters=2, random_state=0).fit(df_where_img_embs_not_na['price'].values.reshape(-1, 1))
  df_where_img_embs_not_na["price_cluster"] = kmeans.labels_

  exp_df = df_where_img_embs_not_na[df_where_img_embs_not_na['price_cluster'] == 1]
  cheap_df = df_where_img_embs_not_na[df_where_img_embs_not_na['price_cluster'] == 0]
  print(f"\n\nexp_df len: {len(exp_df)}, cheap_df len: {len(cheap_df)}")

  for img_embs_not_na_df in [exp_df, cheap_df]:
    img_embs = list(img_embs_not_na_df['img_emb'])


    # img_embs = list(img_embs_not_na_df['img_emb']) #Create a list to compute cosine sim

    #4. compute cosine similarity among all products images in the subset df
    cos_sim_img = util.cos_sim(img_embs, img_embs)
    # print("Cosine-Similarity img:", cos_sim_img)

    #5. Obtain top N similar instances
    # num = 13 # includes the item itself so we take 12+1 
    num = min(13, len(exp_df), len(cheap_df))
    topN_sim_idx = np.argpartition(cos_sim_img, -num, axis=1)[:, -num:]

    #6. Add 12 cols for the top 12 most similar products 
    topN_sim_classes = [[x for x in topN_sim_idx[row] if x != row] for row in range(len(topN_sim_idx))]
    topN_sim_classes_arr = np.asarray(topN_sim_classes)
    print(f"\n{topN_sim_classes_arr.shape}")
    if topN_sim_classes_arr.shape[1] < 12:
      nan_col = np.empty((topN_sim_classes_arr.shape[0],12-topN_sim_classes_arr.shape[1]))
      nan_col.fill(np.nan)
      topN_sim_classes_arr = np.c_[topN_sim_classes_arr,nan_col]
      print(f"\nReshaping with NaNs to accomodate for less # of products in category: {item_group}")
      print(topN_sim_classes_arr)


    img_embs_not_na_df = img_embs_not_na_df.assign(sim_1=topN_sim_classes_arr[:,-1], sim_2=topN_sim_classes_arr[:,-2], sim_3=topN_sim_classes_arr[:,-3], sim_4=topN_sim_classes_arr[:,-4], sim_5=topN_sim_classes_arr[:,-5], sim_6=topN_sim_classes_arr[:,-6], \
                                                  sim_7=topN_sim_classes_arr[:,-7], sim_8=topN_sim_classes_arr[:,-8], sim_9=topN_sim_classes_arr[:,-9], sim_10=topN_sim_classes_arr[:,-10], sim_11=topN_sim_classes_arr[:,-11], sim_12=topN_sim_classes_arr[:,-12])
    
    try:
      # img_embs_not_na_df.assign(sim_1_prod=list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']]))
      img_embs_not_na_df['sim_1_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_1']])
    except:
      img_embs_not_na_df['sim_1_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_2_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_2']])
    except:
      img_embs_not_na_df['sim_2_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_3_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_3']])
    except:
      img_embs_not_na_df['sim_3_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_4_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_4']])
    except:
      img_embs_not_na_df['sim_4_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_5_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_5']])
    except:
      img_embs_not_na_df['sim_5_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_6_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_6']])
    except:
      img_embs_not_na_df['sim_6_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_7_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_7']])
    except:
      img_embs_not_na_df['sim_7_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_8_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_8']])
    except:
      img_embs_not_na_df['sim_8_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_9_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_9']])
    except:
      img_embs_not_na_df['sim_9_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_10_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_10']])
    except:
      img_embs_not_na_df['sim_10_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_11_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_11']])
    except:
      img_embs_not_na_df['sim_11_prod'] = np.nan
    try:
      img_embs_not_na_df['sim_12_prod'] = list(img_embs_not_na_df['article_id'].iloc[img_embs_not_na_df['sim_12']])
    except:
      img_embs_not_na_df['sim_12_prod'] = np.nan

    #7. Merge the new 12 cols with original article_df
    # article_df = pd.merge(article_df,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', left_index=True, right_index=True, how="outer", suffixes=('', '_y'))
    # article_df_new = pd.merge(article_df_new,img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']],on='article_id', suffixes=('', '_y'))
    # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

    # article_df_new = article_df_new.merge(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']], left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
    # article_df_new.drop(article_df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
    appended_data.append(img_embs_not_na_df[['article_id', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']])


  del img_embs_not_na_df
  gc.collect()

  0%|          | 0/15 [00:00<?, ?it/s]

Blazer
105542
1110



100%|██████████| 1110/1110 [00:00<00:00, 312650.42it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()

100%|██████████| 1110/1110 [00:00<00:00, 328721.14it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




exp_df len: 145, cheap_df len: 952

(145, 12)


  7%|▋         | 1/15 [00:00<00:12,  1.13it/s]

...

Vest top
105542
2991



100%|██████████| 2991/2991 [00:00<00:00, 468347.77it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()

100%|██████████| 2991/2991 [00:00<00:00, 377570.68it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




exp_df len: 2172, cheap_df len: 799

(2172, 12)

(799, 12)


100%|██████████| 15/15 [00:33<00:00,  2.22s/it]

time: 33.5 s (started: 2022-05-04 10:03:30 +00:00)


In [ ]:
article_df_new = pd.concat(appended_data)
print(len(article_df_new))

103702
time: 18.8 ms (started: 2022-05-04 10:04:04 +00:00)


In [ ]:
article_df_merged_with_sim_and_prices = article_df.merge(article_df_new, on='article_id', how='outer')

time: 194 ms (started: 2022-05-04 10:10:54 +00:00)


In [ ]:
for col in article_df_merged_with_sim_and_prices.columns:
  print(col)

article_id
product_code
prod_name
product_type_no
product_type_name
product_group_name
graphical_appearance_no
graphical_appearance_name
colour_group_code
colour_group_name
perceived_colour_value_id
perceived_colour_value_name
perceived_colour_master_id
perceived_colour_master_name
department_no
department_name
index_code
index_name
index_group_no
index_group_name
section_no
section_name
garment_group_no
garment_group_name
detail_desc
sim_1_prod
sim_2_prod
sim_3_prod
sim_4_prod
sim_5_prod
sim_6_prod
sim_7_prod
sim_8_prod
sim_9_prod
sim_10_prod
sim_11_prod
sim_12_prod
time: 4.57 ms (started: 2022-05-04 10:26:41 +00:00)


In [ ]:
article_df_merged_with_sim_and_prices.to_csv("/content/drive/MyDrive/CSCI_567_ML/Data/articles_with_top12_similar_images_based_on_price.csv")

time: 2.43 s (started: 2022-05-04 10:10:55 +00:00)


## Exploring articles based on product similarity with prices

In [ ]:
article_sim_df = pd.read_csv("/content/drive/MyDrive/CSCI_567_ML/Data/articles_with_top12_similar_images_based_on_price.csv")

time: 2.79 s (started: 2022-05-04 22:47:22 +00:00)


In [ ]:
print(len(article_sim_df))

105542
time: 2.88 ms (started: 2022-05-04 22:47:25 +00:00)


In [ ]:
for col in article_sim_df.columns:
  print(col)

Unnamed: 0
article_id
product_code
prod_name
product_type_no
product_type_name
product_group_name
graphical_appearance_no
graphical_appearance_name
colour_group_code
colour_group_name
perceived_colour_value_id
perceived_colour_value_name
perceived_colour_master_id
perceived_colour_master_name
department_no
department_name
index_code
index_name
index_group_no
index_group_name
section_no
section_name
garment_group_no
garment_group_name
detail_desc
sim_1_prod
sim_2_prod
sim_3_prod
sim_4_prod
sim_5_prod
sim_6_prod
sim_7_prod
sim_8_prod
sim_9_prod
sim_10_prod
sim_11_prod
sim_12_prod
time: 7.7 ms (started: 2022-05-04 22:47:25 +00:00)


In [ ]:
for col in transaction_df.columns:
  print(col)

t_dat
customer_id
article_id
price
sales_channel_id
time: 1.61 ms (started: 2022-05-04 22:47:25 +00:00)


In [ ]:
min_price_item = transaction_df.nlargest(2,['price'])['article_id']
print(min_price_item.iloc[0])

print(article_sim_df[article_sim_df['article_id'] == min_price_item.iloc[0]])
print(article_sim_df[article_sim_df['article_id'] == min_price_item.iloc[1]])

In [ ]:
# pd.set_option('display.float_format', '{:.4f}'.format)
# transaction_df.describe()['price']

In [ ]:
# transactions_byid = transaction_df.groupby('customer_id').count()

In [ ]:
# transactions_byid.sort_values(by='price', ascending=False)['price'][:10]

In [ ]:
articles_for_merge = article_sim_df[['article_id', 'prod_name', 'product_type_name', 'product_group_name', 'index_name', 'colour_group_name', 'detail_desc', 'sim_1_prod', 'sim_2_prod', 'sim_3_prod', 'sim_4_prod', 'sim_5_prod', 'sim_6_prod', 'sim_7_prod', 'sim_8_prod', 'sim_9_prod', 'sim_10_prod', 'sim_11_prod', 'sim_12_prod']]

time: 50.9 ms (started: 2022-05-04 22:47:25 +00:00)


In [ ]:
articles_for_merge = transaction_df[['customer_id', 'article_id', 'price', 't_dat']].merge(articles_for_merge, on='article_id', how='left')

time: 18.7 s (started: 2022-05-04 22:47:25 +00:00)


In [ ]:
# articles_for_merge = articles_for_merge.merge(article_df_new, on='article_id', how='outer')

time: 725 µs (started: 2022-05-04 10:42:38 +00:00)


In [ ]:
articles_for_merge.head()

,customer_id,article_id,price,t_dat,prod_name,product_type_name,product_group_name,index_name,colour_group_name,detail_desc,sim_1_prod,sim_2_prod,sim_3_prod,sim_4_prod,sim_5_prod,sim_6_prod,sim_7_prod,sim_8_prod,sim_9_prod,sim_10_prod,sim_11_prod,sim_12_prod
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,663713001,0.050831,2018-09-20,Atlanta Push Body Harlow,Underwear body,Underwear,Lingeries/Tights,Black,"Lace push-up body with underwired, moulded, padded cups for a larger bust and fuller cleavage. Narrow, adjustable shoulder straps, an opening with a hook-and-eye fastening at the back and a lined gusset with concealed press-studs.",667641001.0,874620001.0,804666001.0,703416001.0,809487001.0,622031002.0,783830001.0,650040001.0,650037001.0,813613001.0,812428001.0,802529001.0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,541518023,0.030492,2018-09-20,Rae Push (Melbourne) 2p,Bra,Underwear,Lingeries/Tights,Light Pink,"Lace push-up bras with underwired, moulded, padded cups for a larger bust and fuller cleavage. Adjustable shoulder straps and a narrow fastening at the back with two pairs of hooks and eyes.",699253004.0,610926002.0,351933041.0,569594001.0,492853004.0,560389011.0,325087044.0,633142004.0,647822002.0,820572005.0,481110019.0,639525002.0
2,00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,505221004,0.015237,2018-09-20,Inca Jumper,Sweater,Garment Upper body,Divided,Pink,"Jumper in rib-knit cotton with hard-worn details, a wide, roll-edge neckline, and gently dropped shoulders.",642155001.0,693243009.0,677956001.0,788237001.0,522374002.0,832819001.0,564358036.0,678942047.0,845427009.0,870518010.0,844932002.0,801123004.0
3,00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,685687003,0.016932,2018-09-20,W YODA KNIT OL OFFER,Sweater,Garment Upper body,Ladieswear,Pink,"V-neck knitted jumper with long sleeves and ribbing around the neckline, cuffs and hem.",708531002.0,568864003.0,636935002.0,675581008.0,690222004.0,782758002.0,636935010.0,516903015.0,725216004.0,794797003.0,543054019.0,636389002.0
4,00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,685687004,0.016932,2018-09-20,W YODA KNIT OL OFFER,Sweater,Garment Upper body,Ladieswear,Dark Green,"V-neck knitted jumper with long sleeves and ribbing around the neckline, cuffs and hem.",676090001.0,911870005.0,903276001.0,674448003.0,893350002.0,825688002.0,532578028.0,696996005.0,592172007.0,762619003.0,519712006.0,906100001.0


time: 52.3 ms (started: 2022-05-04 22:47:44 +00:00)


In [ ]:
print(len(articles_for_merge))

31788324
time: 1.45 ms (started: 2022-05-04 22:47:44 +00:00)


In [ ]:
min_price_item = transaction_df.nsmallest(2,['price'])

time: 1.68 s (started: 2022-05-04 11:51:30 +00:00)
